In [1]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

True

### Understanding Store in LangChain

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

file_path = r"data\V3. Brand Equity Milk - Ipsos - F'2021.pdf"

loader = PyPDFLoader(file_path=file_path)

# by default, we will split by pages with no text_splitter
documents = loader.load_and_split(text_splitter=None)
documents

[Document(page_content="© 2021 Ipsos. All rights reserved. Contains Ipsos' Confidential and Proprietary information and may not be disclosed or repro duced without the prior written consent of Ipsos.\nMILK\nFinal ReportMILK BRAND \nEQUITY TRACKING\nAugust 10th, 2021", metadata={'source': "data\\V3. Brand Equity Milk - Ipsos - F'2021.pdf", 'page': 0}),
 Document(page_content='© Ipsos2‒Methodology\nData collection\nOnline\n2021 Wave : June 11th-July 5th, 2021\nTarget\nMen and Women\nGrocery shopper (all or share)\nPurchase dairy white milk at least once every 2 to 3 months\nSample\nn=1,800\nAge: 18-65\nWomen : 60% | Men: 40%\nProvinces →\nPlease note that our target sample has been weighted in order to reflect Stats Canada population proportions.BC\n2021:274Prairies \n2021: 308ON\n2021: 314QC\n2021: 612Atlantic\n2021: 292\n2019: 309\n2018: 3002019: 303\n2018: 3002019: 303\n2018: 3002019: 597\n2018: 6002019: 288\n2018: 315Previous waves info:\n2018 : \nTotal Sample n=1,815; \nField: June 

### Unsing the PostgresByteStore

In [ ]:
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_postgres import PGVector
from database import COLLECTION_NAME, CONNECTION_STRING
from utils.store import PostgresByteStore
from langchain_postgres import PostgresSaver, PickleCheckpointSerializer
from utils.custom_sql_record_manager import CustomSQLRecordManager
from utils.index_with_ids import index_with_ids


embeddings = OpenAIEmbeddings()
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=CONNECTION_STRING,
    use_jsonb=True,
)

store = PostgresByteStore(CONNECTION_STRING, COLLECTION_NAME)
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore, 
    docstore=store, 
    id_key=id_key,
)

# define record manager
namespace = f"pgvector/{COLLECTION_NAME}"
record_manager = CustomSQLRecordManager(
    namespace, db_url=CONNECTION_STRING
)
record_manager.create_schema()

retriever

In [ ]:
from utils.utils import generate_reproducible_id_by_content

# Add a reproducible unique doc_id to each document's metadata
for position, doc in enumerate(documents):
    doc.metadata["doc_id"] = generate_reproducible_id_by_content(doc.page_content, doc.metadata)

In [ ]:
documents

In [ ]:
# Generate the list of (doc_id, document) tuples from the documents
doc_id_document_tuples = [(doc.metadata["doc_id"], doc, doc.metadata["source"]) for doc in documents]

# Pass the list of tuples to retriever.docstore.conditional_mset
parent_docs_operations = retriever.docstore.conditional_mset(doc_id_document_tuples)

In [ ]:
parent_docs_operations

### Creating Smaller Documents

In [ ]:
from sqlalchemy import create_engine, Column, String, LargeBinary, select, Table, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import JSONB
from langchain.schema.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

separators = ["\n\n", "\n", ".", "?", "!"]

# Initialize the RecursiveCharacterTextSplitter with fixed parameters
child_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    separators=separators
)

# List to store all sub-documents
all_sub_docs = []

# Database connection setup
engine = create_engine(CONNECTION_STRING)
Session = sessionmaker(bind=engine)
session = Session()

# Define table structure
metadata = MetaData()
langchain_pg_embedding = Table(
    'langchain_pg_embedding', metadata,
    Column('id', String, primary_key=True),
    Column('collection_id', String),
    Column('embedding', LargeBinary),
    Column('document', String),
    Column('cmetadata', JSONB)
)

# Create a dictionary to map doc_id to documents
documents_dict = {doc.metadata['doc_id']: doc for doc in documents}

# Sort the parent documents operations to ensure deterministic processing order
parent_docs_operations = sorted(parent_docs_operations, key=lambda x: x[0])

# Iterate through the operations
for doc_id, operation in parent_docs_operations:
    if operation == 'SKIP':
        # Fetch records from langchain_pg_embedding table for SKIP documents
        query = select(
            langchain_pg_embedding.c.id,
            langchain_pg_embedding.c.collection_id,
            langchain_pg_embedding.c.embedding,
            langchain_pg_embedding.c.document,
            langchain_pg_embedding.c.cmetadata
        ).where(
            (langchain_pg_embedding.c.cmetadata['doc_id'].astext == doc_id) &
            (langchain_pg_embedding.c.cmetadata['type'].astext == 'smaller chunk')
        ).order_by(langchain_pg_embedding.c.id)  # Ensure fixed order

        result = session.execute(query).fetchall()

        # Recreate sub-documents from fetched records
        for row in result:
            metadata = row.cmetadata
            sub_doc_content = row.document
            sub_doc = Document(page_content=sub_doc_content, metadata=metadata)
            all_sub_docs.append(sub_doc)
    elif doc_id in documents_dict:
        # Retrieve the document from the provided documents for non-SKIP documents
        doc = documents_dict[doc_id]
        source = doc.metadata.get("source")  # Retrieve the source from the document's metadata
        sub_docs = child_text_splitter.split_documents([doc])
        # Ensure fixed order for sub-documents
        sub_docs = sorted(sub_docs, key=lambda x: x.page_content)
        for sub_doc in sub_docs:
            sub_doc.metadata["doc_id"] = doc_id  # Assign the same doc_id to each sub-document
            sub_doc.metadata["source"] = f"{source}(smaller chunk)"  # Add the suffix to the source
            sub_doc.metadata["type"] = "smaller chunk"
        all_sub_docs.extend(sub_docs)

# Close the session after use
session.close()

# The resulting sub-documents

# The resulting sub-documents
all_sub_docs

In [ ]:
len(all_sub_docs)

In [ ]:
idx = index_with_ids(all_sub_docs, record_manager, vectorstore, cleanup="incremental",
                                          source_id_key="source")


In [ ]:
idx

### Creating Summaries for Each Parent Chunk

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt_text = """You are an assistant tasked with summarizing and rephrasing text to maintain the original intent of the document. Rephrase the following text chunk in its original language, ensuring to preserve the original meaning and context, even if it comes from a chart or table: \n\n{element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Initialize the Language Model (LLM)
model = ChatOpenAI(temperature=0, model="gpt-4o")

# Define the summary chain
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
from sqlalchemy import create_engine, Column, String, LargeBinary, select, Table, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import JSONB
from langchain.schema.document import Document

# List to store all summary documents
summary_docs = []

# Database connection setup
engine = create_engine(CONNECTION_STRING)
Session = sessionmaker(bind=engine)
session = Session()

# Define table structure
metadata = MetaData()
langchain_pg_embedding = Table(
    'langchain_pg_embedding', metadata,
    Column('id', String, primary_key=True),
    Column('collection_id', String),
    Column('embedding', LargeBinary),
    Column('document', String),
    Column('cmetadata', JSONB)
)

# Create a dictionary to map doc_id to documents
documents_dict = {doc.metadata['doc_id']: doc for doc in documents}

# Collect parent chunks and associated document IDs for documents that are not SKIP and exist in documents_dict
non_skip_docs = [(documents_dict[doc_id], doc_id) for doc_id, operation in parent_docs_operations if operation != 'SKIP' and doc_id in documents_dict]
skip_doc_ids = [doc_id for doc_id, operation in parent_docs_operations if operation == 'SKIP']

# Generate summaries for the parent chunks that are not SKIP
parent_chunk = [doc.page_content for doc, _ in non_skip_docs]
text_summaries = summarize_chain.batch(parent_chunk, {"max_concurrency": 5})
text_summaries_iter = iter(text_summaries)

# Dictionary to store summaries temporarily
temp_summary_docs = {}

# Process non-SKIP documents and store their summaries
for doc, doc_id in non_skip_docs:
    source = doc.metadata.get("source")
    page = doc.metadata.get("page")
    summary_content = next(text_summaries_iter)
    summary_doc = Document(page_content=summary_content, metadata={
        "doc_id": doc_id,
        "source": f"{source}(summary)",
        "page": page,
        "type": "summary"
    })
    temp_summary_docs[doc_id] = summary_doc

# Process SKIP documents and store their summaries
for doc_id in skip_doc_ids:
    query = select(
        langchain_pg_embedding.c.id,
        langchain_pg_embedding.c.collection_id,
        langchain_pg_embedding.c.embedding,
        langchain_pg_embedding.c.document,
        langchain_pg_embedding.c.cmetadata
    ).where(
        (langchain_pg_embedding.c.cmetadata['doc_id'].astext == doc_id) &
        (langchain_pg_embedding.c.cmetadata['type'].astext == 'summary')
    )

    result = session.execute(query).fetchall()

    if not result:
        print(f"No result found for SKIP doc_id {doc_id}")
    else:
        for row in result:
            metadata = row.cmetadata
            summary_content = row.document
            summary_doc = Document(page_content=summary_content, metadata=metadata)
            temp_summary_docs[doc_id] = summary_doc

# Combine the summaries into the final summary_docs list
for doc in documents:
    doc_id = doc.metadata['doc_id']
    if doc_id in temp_summary_docs:
        summary_docs.append(temp_summary_docs[doc_id])
    else:
        # Handle the case where no summary was found or generated
        print(f"No summary found for document ID {doc_id}")

# Close the session after use
session.close()

In [ ]:
summary_docs


In [ ]:
idx = index_with_ids(summary_docs, record_manager, vectorstore, cleanup="incremental",
                                          source_id_key="source")


In [ ]:
idx

### Generating Hypothetical Questions for Each Parent Chunk

In [ ]:
functions = [
    {
        "name": "hypothetical_questions",
        "description": "Generate hypothetical questions",
        "parameters": {
            "type": "object",
            "properties": {
                "questions": {
                    "type": "array",
                    "items": {"type": "string"},
                },
            },
            "required": ["questions"],
        },
    }
]

In [ ]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

question_chain = (
    {"doc": lambda x: x.page_content}
    # Only asking for 5 hypothetical questions, but this could be adjusted
    | ChatPromptTemplate.from_template(
#         """Generate a list of exactly 5 hypothetical questions that the below document could be used to answer, in the original language of the text:\n\n{doc}
# Separate each question with a comma (,)
#         """
"""Générez une liste de 5 questions hypothétiques que le document ci-dessous pourrait être utilisé pour répondre par a des questions sur le document, dans la langue originale du texte. Le document est extrait d'un PPTX ou PDF et peut nécessiter l'analyse du contexte et de la signification. Si le texte est trop court pour générer 5 questions, fournissez autant de questions que le texte le permet.
\nLe texte généré sera utilisé pour effectuer des recherches de similarité dans une base de données vectorielle.
\n\n{doc}\n\n

Séparez chaque question par une virgule (,).
"""
    )
    | ChatOpenAI(max_retries=0, model="gpt-4o").bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

In [ ]:
from sqlalchemy import create_engine, Column, String, LargeBinary, select, Table, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import JSONB
from langchain.schema.document import Document

# List to store all question documents
question_docs = []

# Database connection setup
engine = create_engine(CONNECTION_STRING)
Session = sessionmaker(bind=engine)
session = Session()

# Define table structure
metadata = MetaData()
langchain_pg_embedding = Table(
    'langchain_pg_embedding', metadata,
    Column('id', String, primary_key=True),
    Column('collection_id', String),
    Column('embedding', LargeBinary),
    Column('document', String),
    Column('cmetadata', JSONB)
)

# Create a dictionary to map doc_id to documents
documents_dict = {doc.metadata['doc_id']: doc for doc in documents}

# Separate non-SKIP and SKIP document IDs
non_skip_docs = [(documents_dict[doc_id], doc_id) for doc_id, operation in parent_docs_operations if operation != 'SKIP' and doc_id in documents_dict]
skip_doc_ids = [doc_id for doc_id, operation in parent_docs_operations if operation == 'SKIP']

# Generate hypothetical questions for the parent documents that are not SKIP
parent_documents = [doc for doc, _ in non_skip_docs]

# Debugging: Check the number of parent documents
print(f"Number of non-SKIP documents: {len(parent_documents)}")

# Check if parent_documents is not empty
if parent_documents:
    hypothetical_questions = question_chain.batch(parent_documents, {"max_concurrency": 5})
    hypothetical_questions_iter = iter(hypothetical_questions)
else:
    hypothetical_questions_iter = iter([])

# Dictionary to store questions temporarily
temp_question_docs = {}

# Process non-SKIP documents and store their questions
for doc, doc_id in non_skip_docs:
    source = doc.metadata.get("source")
    page = doc.metadata.get("page")

    try:
        question_list = next(hypothetical_questions_iter)
    except StopIteration:
        print(f"No more questions available for doc_id {doc_id}")
        continue

    for question_content in question_list:
        question_doc = Document(page_content=question_content, metadata={
            "doc_id": doc_id,
            "source": f"{source}(question)",
            "page": page,
            "type": "question"
        })
        if doc_id not in temp_question_docs:
            temp_question_docs[doc_id] = []
        temp_question_docs[doc_id].append(question_doc)

# Process SKIP documents and store their questions
for doc_id in skip_doc_ids:
    query = select(
        langchain_pg_embedding.c.id,
        langchain_pg_embedding.c.collection_id,
        langchain_pg_embedding.c.embedding,
        langchain_pg_embedding.c.document,
        langchain_pg_embedding.c.cmetadata
    ).where(
        (langchain_pg_embedding.c.cmetadata['doc_id'].astext == doc_id) &
        (langchain_pg_embedding.c.cmetadata['type'].astext == 'question')
    )

    result = session.execute(query).fetchall()

    if result:
        questions = []
        for row in result:
            metadata = row.cmetadata
            question_content = row.document
            question_doc = Document(page_content=question_content, metadata=metadata)
            questions.append(question_doc)
        temp_question_docs[doc_id] = questions

# Combine the questions into the final question_docs list
for doc in documents:
    doc_id = doc.metadata['doc_id']
    if doc_id in temp_question_docs:
        question_docs.extend(temp_question_docs[doc_id])

# Close the session after use
session.close()

# The resulting question documents
question_docs


In [ ]:
page_contents = []
for doc in question_docs:
    page_contents.append(doc.page_content)

print(page_contents)

In [ ]:
idx = index_with_ids(question_docs, record_manager, vectorstore, cleanup="incremental",
                                          source_id_key="source")



In [ ]:
idx

### Creating an LCEL Chain and Testing the Retriever

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt template
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("Où doit-on donner son avis ?")


In [ ]:
chain.invoke("Quelle est l'adresse courriel mentionnée dans le document ?")